# MultiPepGen Tutorial: Reproducibilidad de Resultados

Este notebook demuestra cómo utilizar el paquete **MultiPepGen** para cargar un modelo pre-entrenado, generar secuencias de péptidos dirigidas y visualizar sus propiedades físicoquímicas.

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Agregar el path de src si no está instalado como paquete
sys.path.append('../src')

from multipepgen.models.cgan import ConditionalGAN
from multipepgen.utils.descriptors import get_descriptors
from multipepgen.config import LABELS

print("Librerías cargadas exitosamente.")

## 1. Inicialización del Modelo

Configuramos el modelo CGAN con los parámetros por defecto.

In [ ]:
gan = ConditionalGAN(
    sequence_length=35,
    vocab_size=21,
    latent_dim=100,
    num_classes=len(LABELS)
)

# Si tienes pesos pre-entrenados, descomenta la siguiente línea:
# gan.load_model_weights('../models/pretrained/mi_modelo')

print("Modelo inicializado.")

## 2. Generación Masiva de Péptidos

Generaremos 50 péptidos con actividad 'microbiana' y 'bacteriana'.

In [ ]:
target_classes = ['microbiano', 'bacteriano']
num_seqs = 50

generated_df = gan.generate_class(num_seqs, target_classes)
print(f"Generados {num_seqs} péptidos para las clases: {target_classes}")
print(generated_df.head())

## 3. Análisis de Propiedades (Descriptores)

Calcularemos propiedades físicoquímicas para validar la calidad de las secuencias.

In [ ]:
def calculate_properties(df):
    props = []
    for seq in df['sequence']:
        d = get_descriptors(seq)
        props.append(d)
    return pd.DataFrame(props)

properties_df = calculate_properties(generated_df)
print("Propiedades calculadas.")
print(properties_df.describe())

## 4. Visualización de Resultados

Replicamos una figura típica de distribución de Carga Neta e Hidrofobicidad.

In [ ]:
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
sns.histplot(properties_df['NetCharge'], kde=True, color='skyblue')
plt.title('Distribución de Carga Neta')

plt.subplot(1, 2, 2)
sns.histplot(properties_df['Hydrophobicity'], kde=True, color='salmon')
plt.title('Distribución de Hidrofobicidad')

plt.tight_layout()
plt.show()